# Class & Layers & Ensemble

## Class 사용

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [109]:
class Model:
    def __init__(self, sess, name):
        self.sess = sess  # session 선언 앞뒤 변수 선언 모두 session 에 들어옴
        self.name = name  # self.name 으로 변수를 지정해줘야 다른 함수에서도 사용 가능
        self._build_net() # 함수 실행
    
    def _build_net(self, learning_rate=0.01):
        with tf.variable_scope(self.name):
            self.keep_prob = tf.placeholder(tf.float32)
            
            self.x = tf.placeholder(tf.float32, [None, 784])
            x_img = tf.reshape(self.x, [-1,28,28,1])
            self.y = tf.placeholder(tf.float32, [None, 10])
            w1 = tf.Variable(tf.random_normal([3,3,1,10], stddev=0.01))
            b1 = tf.Variable([tf.random.normal([10], stddev = 0.01)])
            l1 = tf.nn.conv2d(x_img, w1, strides=[1,1,1,1],padding="SAME")+b1
            l1 = tf.nn.relu(l1)
            l1 = tf.nn.max_pool(l1,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
            l1 = tf.nn.dropout(l1, keep_prob = self.keep_prob)
            # None, 14, 14, 10
            w2 = tf.Variable(tf.random_normal([3,3,10,15], stddev = 0.01))
            b2 = tf.Variable(tf.random_normal([15], stddev = 0.01))
            l2 = tf.nn.conv2d(l1, w2, strides=[1,1,1,1],padding="SAME") + b2
            l2 = tf.nn.relu(l2)
            self.before_l2 = tf.nn.max_pool(l2,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")
            self.drop_l2 = tf.nn.dropout(self.before_l2, keep_prob = self.keep_prob)
            l2 = tf.reshape(self.drop_l2, [-1, 7*7*15])
            # None, 7, 7, 15
            w3 = tf.get_variable("w3",[7*7*15,10],initializer=
                                tf.contrib.layers.xavier_initializer())
            b3 = tf.get_variable("b3",[10], initializer=
                                tf.contrib.layers.xavier_initializer())
            self.hypothesis = tf.matmul(l2,w3)+b3
            print(self.hypothesis.shape)
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits
                                  (logits=self.hypothesis,labels=self.y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
            #get accuracy
            self.accuracy = tf.reduce_mean(tf.cast(tf.equal
                                (tf.argmax(self.hypothesis,1),tf.argmax(self.y,1)),dtype=tf.float32))
            
    def predict(self,x_test,*y_test, keep_prob = 1):
        return self.sess.run(self.hypothesis, feed_dict={self.x:x_test,self.y:y_test})
    #feed_dict 안에는 placeholder 만 올 수 있음!
    
    def get_accuracy(self, x_test,y_test,keep_prob=1):
        return self.sess.run(self.accuracy, feed_dict={self.x:x_test,
                                    self.y:y_test,self.keep_prob:keep_prob})
        
    def train(self, x_train, y_train, keep_prob=0.7):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.x:x_train,self.y:y_train,
                    self.keep_prob:keep_prob})
        
        

In [116]:
#initialize
sess = tf.Session()
m1 = Model(sess, "rbreff")

sess.run(tf.global_variables_initializer())
                
# parameters
training_epochs = 15
batch_size = 100
num = mnist.train.num_examples//batch_size

for epoch in range(training_epochs):
    avg_cost=0
    for step in range(num):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c,_=m1.train(batch_xs,batch_ys)
        
        avg_cost += c/num 
    print("cost : {:.9f}".format(avg_cost))
    
print(m1.get_accuracy(mnist.test.images, mnist.test.labels))

(?, 10)
cost : 0.379038875


KeyboardInterrupt: 

In [115]:
sess.run(m1.keep_prob)

InvalidArgumentError: You must feed a value for placeholder tensor 'rbff/Placeholder' with dtype float
	 [[node rbff/Placeholder (defined at C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\framework\ops.py:1748) ]]

Original stack trace for 'rbff/Placeholder':
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 664, in launch_instance
    app.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 583, in start
    self.io_loop.start()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 153, in start
    self.asyncio_loop.run_forever()
  Fi

# Layers

- average_pooling1d(...) : Average Pooling layer for 1D inputs.
- average_pooling2d(...) : Average Pooling layer for 2D inputs(images).
- average_pooling3d(...) : Average Pooling layer for 3D inputs(volumes).
- batch_normalization(...) : Functional interfact for the batch normalization layer.
- conv1d(...) : Functional interface for 1D convolution layer (e.g.temporal convolution)
- conv2d(...) : Functional interface for the 2D convolution layer.
- conv2d_transpose(...) : Transposed convolution layer(sometimes called Deconvolution)
- conv3d(...) : Functional interface for the 3D convolution layer.
- dense(...) : Functional interface for the densely-connected layer.
- dropout(...) : Applies Dropout to the input.
- max_pooling1d(...) : Max Pooling layer for 1D inputs.
- max_pooling2d(...) : Max Pooling layer for 2D inputs(images).
- max_pooling3d(...) : Max Pooling layer for 3D inputs(volumes).
- separable_conv2d : Functional interface for the depthwise separable 2D convolution layer.

In [ ]:
conv1 = tf.layers.conv2d(inputs=x_img,filters=32,kernel_size=[3,3],strides=[1,1]
                         padding = "SAME", activation = tf.nn.relu)
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2],strides=2,
                               padding = "SAME")
dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=self.training)
# training 인자가 False 면 자동으로 1, training =True면 rate을 반환,
# default 는 False
conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3],
                        padding="SAME", strides=[1,1], activation = tf.nn.relu)
pool2 = tf.layers.pooling2d(inputs=conv2, pool_size=[2,2],strides=2,
                           padding="SAME")
dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7, training=self.training)

flat = tf.reshape(dropout2, [-1, 128*4*4])   # 이거에 대해선 좀 알아야 함!
dense4 = tf.layers.dense(inputs=flat,units=625,activation=tf.nn.relu)
dropout4 = tf.layers.dropout(inputs=dense4, rate=0.7, training = self.training)
dense5 = tf.layers.dense(inputs=dropout4, units=10, activation=tf.nn.relu)

## Ensemble training

In [ ]:
models = []
num_models = 6
for m in range(num_models):
    modles.append(Model(sess, "model"+str(m))) # 총 6개의 모델 생성
sess.run(tf.global_variables_initializer())
# 모든 model session의 변수 초기화(변수 할당)

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(models))
    total_batch = int(mnist.train.num_examples/batch_size)
    for step in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        
        for m_idx, m in enumerate(models):
            c,_ m.train(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c/ batch_size
        print(f"cost: {avg_cost_list}")

## Ensemble prediction using ensemble sum

In [ ]:
test_size = len(mnist.test.labels) # 행의 개수가 나옴
predictions = np.zeros(test_size*10).reshape(test_size,10)

for m_idx, m in enumerate(models):
    print(m_idx, "Accuracy", m.get_accuracy(mnist.test.images,mnist.test.labels))
    p = m.predict(mnist.test.images)
    predictions += p

ensemble_correct_prediction = tf.equal(
            tf.argmax(predictions,1), tf.argmax(mnist.test.labels,1))
ensemble_accuracy= tf.reduce_mean(tf.cast(ensemble_correct, dtype=tf.float32))

print("Ensemble_correct", sess.run(ensemble_correct_prediction))

In [111]:
len(np.arange(6).reshape(2,3))

2